In [1]:
import pandas as pd
import numpy as np
import re
import sklearn
from nltk.corpus import stopwords #to remove stoprwords: words which doesn't add meaning to context like articles (a/an/the), etc.
from nltk.stem.porter import PorterStemmer #used to perform stemming:to remove the prefix and suffix of the word and return the root word
from sklearn.feature_extraction.text import TfidfVectorizer #to convert text into feature vectors which are nothing but numbers
from sklearn.model_selection import train_test_split #used to divide data into train and test data
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
print(stopwords.words('english')) # words which doesn't add meaning so we remove them

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing

In [4]:
news_data = pd.read_csv('/content/train.csv')

In [5]:
news_data.shape

(20800, 5)

In [6]:
news_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
news_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
news_data = news_data.fillna('')

In [9]:
#merging author and title
news_data['content'] = news_data['author']+' '+news_data['title']

In [11]:
print(news_data['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [12]:
#separating data and label
x = news_data.drop(columns='label', axis=1) #to remove a column we use axis=1 and to remove a row we use axis=0
y = news_data['label']

In [13]:
print(x)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming: process of reducing a word to its root word

example: actor, actress, acting -->
root word: act

In [14]:
porter_stem = PorterStemmer()

In [15]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content) #to remove all special characters and numbers
  stemmed_contect = stemmed_content.lower() #to convert all characters into lower case
  stemmed_content = stemmed_content.split() #to split the words into list
  stemmed_content = [porter_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] #to remove stopwords
  stemmed_content = ' '.join(stemmed_content) #to convert list into string
  return stemmed_content

In [16]:
news_data['content'] = news_data['content'].apply(stemming)

In [17]:
print(news_data['content'])

0        darrel lucu hous dem aid we didn even see come...
1        daniel j flynn flynn hillari clinton big woman...
2           consortiumnew com whi truth might get you fire
3        jessica purkiss civilian kill in singl us airs...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper t i trump poster child for...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci is said...
20798    alex ansari nato russia to hold parallel exerc...
20799                       david swanson what keep f aliv
Name: content, Length: 20800, dtype: object


In [18]:
#separating data and label
x = news_data['content'].values
y = news_data['label'].values

In [19]:
print(x)

['darrel lucu hous dem aid we didn even see comey letter until jason chaffetz tweet it'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com whi truth might get you fire' ...
 'michael j de la merc rachel abram maci is said receiv takeov approach hudson bay the new york time'
 'alex ansari nato russia to hold parallel exercis in balkan'
 'david swanson what keep f aliv']


In [20]:
print(y)

[1 0 1 ... 0 1 1]


In [21]:
y.shape

(20800,)

In [22]:
#converting textual data to numerical data
vector = TfidfVectorizer() #to convert text into feature vectors which are nothing but numbers
vector.fit(x)

x = vector.transform(x)

In [23]:
print(x)

  (0, 16586)	0.19235766814206556
  (0, 16055)	0.29053629972079575
  (0, 15762)	0.25180682754084327
  (0, 13528)	0.22682424464275375
  (0, 8949)	0.3214177525383042
  (0, 8670)	0.25823746120670493
  (0, 7731)	0.219100353547038
  (0, 7639)	0.1611238443347535
  (0, 7040)	0.19336678366028098
  (0, 4998)	0.20625126961544935
  (0, 4028)	0.2667908047240904
  (0, 3811)	0.23915031396884368
  (0, 3619)	0.31814479104571186
  (0, 2977)	0.21821000835755877
  (0, 2501)	0.3250028762214839
  (0, 272)	0.2387684291853839
  (1, 16892)	0.30071745655510157
  (1, 6849)	0.1904660198296849
  (1, 5528)	0.7143299355715573
  (1, 3587)	0.26373768806048464
  (1, 2831)	0.19094574062359204
  (1, 2241)	0.3827320386859759
  (1, 1910)	0.15521974226349364
  (1, 1512)	0.2939891562094648
  (2, 17095)	0.29337976465513754
  :	:
  (20797, 9631)	0.17242189281191916
  (20797, 9561)	0.2918128273796141
  (20797, 9030)	0.35719284755530417
  (20797, 8404)	0.22049990081059304
  (20797, 7611)	0.13993333005918895
  (20797, 7077)	0.215

Splitting the data to train and test

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2) #to split data into train and test data (20% is test and 80% is train). stratify helps to spearate the data in equal proportion of true and false

Training the model: logistic regression model

In [25]:
model = LogisticRegression()

In [26]:
model.fit(x_train, y_train)

LogisticRegression()

Evaluating model using accuracy score

In [27]:
#accuracy score on training data
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train) #to check how accurate the model is

In [28]:
print('Accuracy score of training data : ', training_data_accuracy)

Accuracy score of training data :  0.9868990384615385


In [29]:
#for binary classification it is best to use logistic regression
#accuracy score on training data
x_test_prediction = model.predict(x_test)
testing_data_accuracy = accuracy_score(x_test_prediction, y_test) #to check how accurate the model is

In [30]:
print('Accuracy score of testing data : ', testing_data_accuracy)

Accuracy score of testing data :  0.9762019230769231


Making Predictive System

In [33]:
x_new = x_test[1] #new news to make predictions and as our model has not seen test data we use the first row of test data as new news

prediction = model.predict(x_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [34]:
print(y_test[1])

0
